<a href="https://colab.research.google.com/github/Mary82/cross-Modal-Learning/blob/master/CIFAR100_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import os 
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from google.colab import drive
drive.mount('/content/gdrive') 

os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
from CIFAR100_Utiliy import *

In [2]:
# Load CIFAR100 data

num_classes = 100
(x_train, y_train), (x_test, y_test) = load_data(num_classes)

print "Shape of x_train: ", x_train.shape 
print "Shape of x_test: ", x_test.shape 


Shape of x_train:  (50000, 32, 32, 3)
Shape of x_test:  (10000, 32, 32, 3)


In [0]:

'''
CNN MODEL TO TRAIN CIFAR100

'''
model = Sequential()

model.add(Conv2D(128, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))



In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 128)       147584    
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 256)       295168    
_________________________________________________________________
activation_3 (Activation)    (None, 15, 15, 256)       0         
__________

In [0]:
# Load the latest training weight for model
model.load_weights('Models/CIFAR100_Weight.h5')


In [0]:

# initiate RMSprop optimizer
# for toward the end training
#opt = keras.optimizers.adam(lr=0.00000005, decay=1e-3,beta_1 = 0.9, beta_2 = 0.95,amsgrad=True)
opt = keras.optimizers.rmsprop(lr = .0001)
# Train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

  
  


In [0]:
#data augmentation

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train) 



In [13]:
'''
Train the model 
checkpoint : save the network weights when there is an improvement in accuracy
'''

filepath="Models/weights-improvement-{epoch:02d}-{val_acc:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint,LearningRateScheduler(lr_schedule)]


batch_size = 128
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=10000,\
                    verbose=0,validation_data=(x_test,y_test), shuffle = True)

  


In [14]:

#testing
scores = model.evaluate(x_train, y_train, batch_size=128, verbose=1)
print('\nTrain result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


50000/50000 [==============================] - 19s 385us/step

Train result: 99.466 loss: 0.020
10000/10000 [==============================] - 4s 385us/step

Test result: 67.920 loss: 1.772


In [0]:
model_json = model.to_json()
with open('cifar_model.json', 'w') as json_file:
  json_file.write(model_json)